# Steal The Files - Documentation (EN)
Client | Server | Core - JavaFX 2D platformer

**Contents**
1. Overview & Architecture
2. Setup & Start
3. Game Logic: Steps, Decisions, Notes
4. UI/Navigation: Steps, Decisions, Notes
5. Level Build: Steps, Decisions, Notes
6. Audio & Settings
7. Networking & Data
8. Debugging & Open Points
9. Roadmap Nuggets

## 1. Overview & Architecture
- Goal from code: grab USB -> unlock file collection -> collect folders -> survive hazards -> touch finish.
- Modules: **core** (entities, events, packets/serialization), **client** (JavaFX screens, levels, audio, input), **server** (TCP listener, packet routing, DB hooks prepared).
- Startup path: `Game.start` loads `config.json`, applies sound (muted/volume), registers listeners, builds `ScreenManager`, creates MainMenu/Settings, selects level via `currentLevel` (Tutorial/Second/Boss).
- Game loop: dedicated thread in `ScreenManager` (~200 Hz) computes `delta`+FPS, invokes `currentScreen.update(delta)` via `Platform.runLater`.
- Runtime pipeline:
```text
Scene input -> InputManager -> ScreenManager.update(delta) -> GameScreen.update(delta)
  -> physics/collision -> camera offset -> HUD update -> render/sound
```

## 2. Setup & Start
- Requirements: JDK 19, JavaFX 17, Maven; assets under `/assets`.
- Build all modules:
```bash
mvn clean install
```
- Run (IDE): Server `de.cyzetlc.hsbi.Server` (TCP 25570), Client `de.cyzetlc.hsbi.game.Game` (connects to localhost, shows Loading -> MainMenu).
- Key config fields: `currentLevel`, `soundVolume` (0-1), `soundMuted`, `messages.game.prefix`, `mysql` block (not auto-wired in client), `tutorialFinished`.

## 3. Game Logic: Steps, Decisions, Notes
**Step-by-step (GameScreen + Entities/Blocks)**
- Capture input: `InputManager` tracks held keys + `pollJustPressed` for toggles.
- Player init: `EntityPlayer` (maxHealth=3) reset on screen switch; name tag & sprite placed.
- Movement params: gravity=15, moveSpeed=450, jumpPower=800 (Jump/Speed perks +25% for 10s).
- Update loop: compute dx/dy, apply gravity, build next bounds.
- Collisions: platforms first (landing/sides), then blocks (items/perks/hazards/enemy); `pendingBlocks` for freshly spawned lasers.
- Interaction: E triggers `GasBarrierBlock.deactivate()` only if flipper flag set.
- Enemies: `RobotEnemyBlock` patrols within [minX,maxX], faces player, fires `LaserBlock` if |dx|<440 (cooldown 1.2s, lifetime 4s, 1 dmg; removed even in GodMode). Stomp from above kills.
- Hazards: `LavaBlock` -0.25 HP/tick (ignored in GodMode).
- Items/Progress: `USBStickBlock` sets `canCollectFiles=true`; `FolderBlock` counts only then and deactivates; `FlipperItem` sets barrier key flag.
- Camera: dead zone marginX=400, marginY=150, smoothing=0.1; sprites offset by camera.
- HUD: quest text, files `X/Y` (active folders), flipper hint, health hearts (maxHealth=3), debug bar on F2, tooltips on F1.
- Game states: game over on HP<=0 or fall -> MainMenu; pause shows overlay (logic keeps running).

**Design decisions**
- Delta-based physics in own thread, UI updates on JavaFX thread to avoid stutter.
- F3 toggles GodMode/NoClip for fast debugging; GodMode blocks damage, NoClip skips collisions.
- Fixed laser lifetime (4s) for simple timing.
- Folder collection gated by USB for clear progression.
- Fixed camera smoothing/dead-zone to keep motion stable.

**Tech notes / issues**
- Pause does not halt logic, only shows overlay.
- `Level.update()` empty; movers rely on block/entity update.
- LevelFinished stats are hardcoded (time/score/kills/HP loss).
- No per-level camera bounds (can scroll off-level).

## 4. UI/Navigation: Steps, Decisions, Notes
**Step-by-step build**
- LoadingScreen: animated background, 3s timeline bar, loads `Material` textures async, then MainMenu.
- MainMenuScreen: parallax background (two sliding images), buttons Start/Multiplayer/Settings/Exit, dummy achievements, footer with version/copyright.
- SettingsScreen: top-right panel (220x90), slider 0-1, mute button; persists directly to `config.json`, disables slider when muted.
- MultiplayerScreen: three static server cards (slot/MOTD), back button.
- GameScreen: clear root, draw background, place player, draw level (platforms/blocks), buttons Back/Pause, HUD (quest, files, optional debug, health bar), pause overlay.
- LevelFinishedScreen: translucent overlay, stats text (time/score/kills/lives lost hardcoded), buttons for next level (if any) or main menu.

**Design decisions**
- Fullscreen, undecorated stage, fullscreen exit disabled for immersion.
- Buttons/HUD on top-left; health on top-right to keep playfield clear.
- Dummy achievements/server list kept minimal while layout is defined.
- Pause overlay only, to observe logic while debugging.

**Tech notes / issues**
- Pause overlay lacks logic halt.
- LevelFinished shows placeholder stats.
- Fixed fullscreen size, no resize handling.
- All UI updates via JavaFX thread (`Platform.runLater`).

## 5. Level Build: Steps, Decisions, Notes
**TutorialLevel**
- Platforms tiered; lava fills gaps (`placeLavaBetweenPlatforms`).
- Blocks: early USB, JumpBoost, SpeedBoost, ServerBlock, FolderBlock, FloatingPlatform linking islands, Finish on the right.
- `onFinish`: sets `tutorialFinished=true`, saves config.
- Next level: SecondLevel.

**SecondLevel**
- Many platforms incl. airborne, 8 FolderBlocks, FlipperItem near start, JumpBoost mid, RobotEnemy on final platform, Finish far right; GasBarrier prepared (commented).
- One-time spawn sound on entry.
- Next level: BossLevel.

**BossLevel**
- Wide start platform, four RobotEnemyBlocks in a lane, Finish beyond; no next level.

**Design decisions**
- USB before folders to enforce progression.
- Early FlipperItem in SecondLevel to enable barrier mechanic if activated.
- Mix of floating platform + enemy for variety without complex physics.

**Tech notes / issues**
- Level coordinates hardcoded; no external editor/JSON.
- `Level.update()` unused for global ticks.
- No camera bounds per level.
- GasBarrier in SecondLevel currently commented.


## 6. Audio & Settings
- `SoundManager`: media caching, global volume 0-1, mute flag, dB boost factor, background music via `playBackground(Music.MENU, true)`, ducking for SFX (`playWithDuck`).
- SettingsScreen syncs slider/mute with SoundManager and writes immediately to config.
- SFX in code: pickup/interaction clicks, buff sounds (Jump/Speed), USB, laser/robot; tutorial checks `tutorialFinished` to reduce noise.
- LoadingScreen prewarms Material textures to reduce first-draw stutter.

## 7. Networking & Data
- Client: `connectAndRun` -> TCP localhost:25570, sends `ClientLoginPacket` (UUID), starts receiver/sender.
- Sender: `ClientDataPacket` heartbeat every 50ms.
- Server: accepts sockets, spawns `MultiClientHandler`, sends periodic `UserMessagePacket("Connection is stable")`, processes packets via `ReceivePacketEvent` (cancelable).
- Packets/serialization live in **core** (`SerializationUtils`, `Packet`, concrete packet classes).
- DB hook present (`QueryHandler`, `MySQLCredentials`), connection disabled by default.
- Score sending: `sendFinalScore(long)` with `ClientSubmitScorePacket` (level name hardcoded "Level_1").

## 8. Debugging & Open Points
- Live toggles: F1/F2/F3, pause overlay (logic continues), health bar reacts, files counter after USB unlock.
- Manual checks from code: pick USB -> collect folder -> finish triggers LevelFinished; Jump/Speed buffs run 10s and reset; RobotEnemy stompable, lasers expire after 4s; Flipper + GasBarrier (if active) disabled via E.
- Open items: real pause (stop update loop), use `Level.update()` for movers/timers, real stats in LevelFinished, camera clamps per level, enable GasBarrier in SecondLevel, finalize multiplayer/highscore with DB.


## 9. Roadmap Nuggets
- Short term: real pause, real LevelFinished stats, camera bounds, enable GasBarrier path in SecondLevel.
- Mid term: use Level.update for global ticks/movers, externalizable levels (JSON/editor), HUD timer/score.
- Long term: complete multiplayer/highscore DB flow, hook achievements/server list to backend.
